In [2]:
import pandas as pd

categories_dict = {'Koła widokowe': 'ferris_wheels',
                  'Areny sportów zimowych': 'winter_sports',
                  'Świątynie hinduizmu': 'hindu_temples',
                  'Obiekty archeologiczne': 'archaeology',
                  'Kopce/kurchany': 'tumuluses',
                  'Muzea biograficzne': 'biographical_museums',
                  'Muzea modowe': 'fashion_museums',
                  'Parki rozrywki': 'amusement_parks',
                  'Parki wodne': 'water_parks',
                  'Parki miniatur': 'miniature_parks',
                  'Baseny, termy i sauny': 'baths_and_saunas',
                  'Ścianki wspinaczkowe': 'climbing',
                  'Stadiony': 'stadiums',
                  'Źródła': 'natural_springs',
                  'Rzeki, kanały, wodospady': 'water',
                  'Rezerwaty przyrody': 'nature_reserves',
                  'Plaże': 'beaches',
                  'Stacje kolejowe': 'railway_stations',
                  'Zapory': 'dams',
                  'Mennice': 'mints',
                  'Kopalnie': 'mineshafts',
                  'Muzea nauki i techniki': 'science_museums',
                  'Kościoły': 'churches',
                  'Katedry': 'cathedrals',
                  'Klasztory': 'monasteries',
                  'Synagogi': 'synagogues',
                  'Meczety': 'mosques',
                  'Zamki': 'castles',
                  'Wieże obronne': 'fortified_towers',
                  'Bunkry': 'bunkers',
                  'Muzea militarne': 'military_museums',
                  'Pola bitew': 'battlefields',
                  'Cmentarze wojenne': 'war_graves',
                  'Cmentarze': 'cemeteries',
                  'Mauzolea': 'mausoleums',
                  'Krypty': 'crypts',
                  'Murale': 'wall_painting',
                  'Fontanny': 'fountains',
                  'Rzeźby': 'sculptures',
                  'Zieleń miejska': 'gardens_and_parks',
                  'Muzea archeologiczne': 'archaeological_museums',
                  'Galerie sztuki': 'art_galleries',
                  'Muzea historyczne': 'history_museums',
                  'Muzea lokalne': 'local_museums',
                  'Muzea narodowe': 'national_museums',
                  'Planetaria': 'planetariums',
                  'Zoo': 'zoos',
                  'Akwaria': 'aquariums',
                  'Drapacze chmur': 'skyscrapers',
                  'Wieże (zegarowe, widokowe)': 'towers',
                  'Budynki historyczne': 'historic_architecture',
                  'Mosty': 'bridges',
                  'Pomniki': 'monuments'}

def read_form_result(csv_path):
  df = pd.read_csv(csv_path)
  df = df.drop(df.columns[-6:], axis=1)
  df = df.drop(df.columns[0], axis=1)
  cols = df.columns

  new_col_names = ['text', 'date']
  for col in cols[3:]:
    new_col_names_cat = [col.split('[')[-1][:-1] for col in cols[2:]]

  new_col_names += new_col_names_cat
  df.columns = [categories_dict[cat] if cat in categories_dict else cat for cat in new_col_names]

  return df

In [ ]:
df = read_form_result('./wibit_form.csv')
df

In [3]:
! pip install stop_words

In [38]:
import re
from string import punctuation
import nltk
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
from stop_words import get_stop_words

stopwords_pl = get_stop_words("pl")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
def preprocess_text(text):
    translator = str.maketrans("", "", punctuation)
    new_text = text.translate(translator)
    new_text = re.sub(r'\d+', '', new_text)
    new_text = new_text.lower()
    new_text = re.sub(r'\s+', ' ', new_text)
    tokens = new_text.split(' ')

    filtered_tokens = [token for token in tokens if token not in stopwords_pl]
    filtered_tokens = [token for token in filtered_tokens if token!= '']
    processed_text = " ".join(filtered_tokens)

    return processed_text

In [40]:
preprocess_text(df['text'][1])

'interesuję wszystkim związane z przyrodą zwierzętami i roślinami lubię miejsca otoczony takimi rzeczami lubię piesze wycieczki i aktywne zwiedzanie nowych miejsc lubię restauracje oferujące sushi i inne azjatyckie potrawy'

In [ ]:
stopwords_pl

In [ ]:
! python -m spacy download pl_core_news_sm

In [21]:
import torch
import spacy

spcay_nlp = spacy.load("pl_core_news_sm")

In [41]:
pre_prep_texts = pd.Series([preprocess_text(df['text'][i]) for i in range(len(df))])
pre_prep_texts[:3]

0    moimi głównymi zainteresowaniami nowe technolo...
1    interesuję wszystkim związane z przyrodą zwier...
2    interesuję ogrodnictwem planując wyjazdy turys...
dtype: object

In [42]:
texts_spacy = pre_prep_texts.map(spcay_nlp)

lemmatized_texts = texts_spacy.map(lambda doc: [t.lemma_ for t in doc])
lemmatized_texts[:3]

0    [mój, główny, zainteresowania, nowy, technolog...
1    [interesować, wszyscy, związać, z, przyroda, z...
2    [interesować, ogrodnictwo, planuć, wyjazd, tur...
dtype: object

In [43]:
lemmatized_texts[1]

['interesować',
 'wszyscy',
 'związać',
 'z',
 'przyroda',
 'zwierzętami',
 'i',
 'roślina',
 'lubić',
 'miejsce',
 'otoczć',
 'taki',
 'rzecz',
 'lubić',
 'piesze',
 'wycieczka',
 'i',
 'aktywny',
 'zwiedzać',
 'nowy',
 'miejsce',
 'lubić',
 'restauracja',
 'oferuć',
 'sushi',
 'i',
 'inny',
 'azjatycki',
 'potrawa']

In [44]:
spacy_vectors = texts_spacy.map(lambda doc: doc.vector)
spacy_vectors[:3]

0    [0.22857785, 0.87393624, 1.3016139, -0.6452852...
1    [0.7168031, 0.5955319, 2.009072, -0.37234628, ...
2    [1.576092, -0.039090604, 1.3596128, 0.8685283,...
dtype: object

In [45]:
len(spacy_vectors[0])

96

In [46]:
len(spacy_vectors[1])

96

In [47]:
spacy_vectors[1]

array([ 0.7168031 ,  0.5955319 ,  2.009072  , -0.37234628,  0.82128763,
       -0.35048795,  0.24421151,  0.39252314,  1.8342274 , -0.6283602 ,
       -0.16024429, -0.0796274 , -1.6359597 ,  0.17821568,  1.7413445 ,
       -0.5667683 ,  0.2629507 , -1.0147601 ,  0.11391503,  0.26811373,
        1.294093  , -0.09958805, -0.00599426, -0.21897136, -1.1244534 ,
       -1.2906537 , -1.513387  , -0.29813954,  0.69212586,  0.7402044 ,
        0.2726266 ,  0.19464204,  0.3574349 , -0.58347213,  1.5331676 ,
       -1.1687576 ,  0.20947945, -1.3128127 ,  0.15884131, -0.9578256 ,
       -0.02474066,  0.7200645 , -0.426533  ,  0.26914862,  0.46717167,
       -0.85749143, -2.1566858 , -1.4873819 ,  1.3507147 ,  0.22768906,
       -1.1716539 , -0.03886688, -0.5212391 , -0.80810887, -1.6326982 ,
       -0.5193767 , -0.14037827, -0.126084  , -1.0485944 ,  0.77830195,
        0.09384193,  0.76963216,  0.75011593, -0.16475569,  1.8674141 ,
        0.6732757 ,  0.4142965 , -0.23759443,  0.67565805,  0.18

In [ ]:
# spacy returns 96 element vectors of texts - might be useful in future tasks

In [ ]:
! pip install googletrans==4.0.0-rc1

In [5]:
from googletrans import Translator

translator = Translator()

df['text_en'] = [translator.translate(sentence, src='pl', dest='en').text for sentence in df['text']]

In [7]:
df[['text', 'text_en']].head(5)

,text,text_en
0,Moimi głównymi zainteresowaniami są nowe techn...,New technologies and scientific issues are my ...
1,Interesuję się wszystkim co związane z przyrod...,I am interested in everything related to natur...
2,"Interesuję się ogrodnictwem, dlatego planując ...","I am interested in gardening, which is why whe..."
3,Interesuję się sportem i muzyką. Moje hobby to...,I am interested in sport and music.My hobby is...
4,"Wypoczynek nad wodą, interesuje się lotnictwem...","Remaining by the water, is interested in aviat..."


In [19]:
# https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base
# in colab check if GPU environmet is checked

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [20]:
paraphrase(df['text_en'][1],
           num_return_sequences=3,
           no_repeat_ngram_size=3)

['I am fascinated by the natural world, particularly animals and plants. I enjoy traveling and exploring new places, as well as dining out at restaurants serving sushi and other Asian cuisines.',
 'The pursuit of nature, animals, and plants fascinates me, which is why I enjoy traveling to places that are surrounded by them. I have a passion for exploring the outdoors and experiencing new places, as well as dining out at restaurants serving sushi and other Asian culinary specialties.',
 'My fascination lies in exploring the natural world, encountering nature, and observing plants and animals, which is why I enjoy visiting places with great excitement. I relish traveling and engaging new experiences, as well as enjoying Japanese-style sushi and other Asian culinary treats.']

In [21]:
paraphrases_en = [paraphrase(text, num_return_sequences=3, no_repeat_ngram_size=3) for text in df['text_en']]

paraphrases_en[1]

['I am fascinated by the natural world, particularly animals and plants. I enjoy traveling and exploring new places, as well as dining out at restaurants serving sushi and other Asian cuisines.',
 'The pursuit of nature, animals, and plants fascinates me, which is why I enjoy traveling to places that are surrounded by them. I have a passion for exploring the outdoors and experiencing new places, as well as dining out at restaurants serving sushi and other Asian culinary specialties.',
 'My fascination lies in exploring the natural world, encountering nature, and observing plants and animals, which is why I enjoy visiting places with great excitement. I relish traveling and engaging new experiences, as well as enjoying Japanese-style sushi and other Asian culinary treats.']

In [23]:
paraphrases_pl = []

for paraph_texts_en in paraphrases_en:
  paraph_texts_pl = [translator.translate(sentence, src='en', dest='pl').text for sentence in paraph_texts_en]
  paraphrases_pl.append(paraph_texts_pl)

paraphrases_pl[1]

['Fascynuje mnie świat przyrody, zwłaszcza zwierzęta i rośliny.Lubię podróżować i odkrywać nowe miejsca, a także jeść w restauracjach serwujących sushi i inne kuchnie azjatyckie.',
 'Fascynuje mnie dążenie do przyrody, zwierząt i roślin, dlatego lubię podróżować do miejsc, które są ich otoczone.Mam pasję do odkrywania na świeżym powietrzu i doświadczania nowych miejsc, a także jedzących w restauracjach serwujących sushi i innych azjatyckich specjalności kulinarnych.',
 'Moja fascynacja polega na odkrywaniu świata przyrody, spotykaniu natury i obserwowaniu roślin i zwierząt, dlatego lubię odwiedzać miejsca z wielkim podnieceniem.Podobno podróżowanie i angażowanie nowych doświadczeń, a także cieszę się sushi w stylu japońskim i innych azjatyckich smakołyków kulinarnych.']

In [36]:
import random
import re

def shuffle_sentences(text):
    sentences = text.split('.')
    random.shuffle(sentences)
    shuffled_text = '.'.join(sentences)
    return shuffled_text


In [37]:
paraphrases_pl_shfl = []

for paraph_texts_pl in paraphrases_pl:
  paraph_texts_pl_shfl = [shuffle_sentences(text) for text in paraph_texts_pl]
  paraphrases_pl_shfl.append(paraph_texts_pl_shfl)

paraphrases_pl_shfl[1]

['.Fascynuje mnie świat przyrody, zwłaszcza zwierzęta i rośliny.Lubię podróżować i odkrywać nowe miejsca, a także jeść w restauracjach serwujących sushi i inne kuchnie azjatyckie',
 'Fascynuje mnie dążenie do przyrody, zwierząt i roślin, dlatego lubię podróżować do miejsc, które są ich otoczone..Mam pasję do odkrywania na świeżym powietrzu i doświadczania nowych miejsc, a także jedzących w restauracjach serwujących sushi i innych azjatyckich specjalności kulinarnych',
 'Podobno podróżowanie i angażowanie nowych doświadczeń, a także cieszę się sushi w stylu japońskim i innych azjatyckich smakołyków kulinarnych.Moja fascynacja polega na odkrywaniu świata przyrody, spotykaniu natury i obserwowaniu roślin i zwierząt, dlatego lubię odwiedzać miejsca z wielkim podnieceniem.']

In [43]:
df_row = df.iloc[1]
df_row['text'] = 'ala ma kota'

<ipython-input-43-189419c0fdb1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_row['text'] = 'ala ma kota'


In [46]:
type(df_row)

pandas.core.series.Series

In [ ]:
oversampled_df_arr = []

for i in range(len(df)):
  for j in range(3):
    df_row = df.iloc[i]
    df_row['text'] = paraphrases_pl_shfl[i][j]
    oversampled_df_arr.append(df_row)

oversampled_df = pd.DataFrame(oversampled_df_arr)
oversampled_df = oversampled_df.reset_index(drop=True)
oversampled_df.head(5)

In [ ]:
random.shuffle(oversampled_df_arr)
oversampled_df = pd.DataFrame(oversampled_df_arr)
oversampled_df = oversampled_df.reset_index(drop=True)
oversampled_df.head(5)

In [ ]:
oversampled_df = oversampled_df.drop(['text_en'], axis=1)
oversampled_df

In [57]:
oversampled_df.to_csv('wibit_form_oversampled.csv')